In [1]:
def daytype2number(x):
    return {
        'normal': 0,
        'partial': 1,
        'off': 2,
    }[x]

In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import datetime, time

data_dir ='/data/CS120/'
weather_dir ='/data/CS120Weather/'
csv_file = '../CS120/general/timezones.csv'
# data_dir = '../../../Data/depression2016/CS120Data/CS120/'
# csv_file = '../../../Data/depression2016/CS120Data/timezones.csv'
# weather_dir ='../../../Data/depression2016/CS120Weather/'

subjects = os.listdir(data_dir)
timezones = pd.read_csv(csv_file,sep='\t',header=None)

In [7]:
# subjects = subjects[:2]

ind_nan = np.where(np.isnan(timezones[1]))[0]
timezones.loc[ind_nan,1]=0

acts = []
emas = []
emss = []
wtrs =[]
subjects_edited = []

for (cnt,subj) in enumerate(subjects):
    
    print(cnt,subj,)
    
    if subj=='1515656':
        print('subject skipped due to spoofed gps data')
        continue
    
    subjects_edited.append(subj)
    
    act = pd.DataFrame(columns=['date','act'], dtype=float)
    if os.path.exists(data_dir+subj+'/act.csv'):
        data = pd.read_csv(data_dir+subj+'/act.csv',sep='\t',header=None)
        # convert timestamps to daystamps
        data[0] = np.floor((data[0]+ 3600*float(timezones.loc[timezones[0]==subj,1]))/86400.0)
        # loading into new matrix
        act['date'] = np.arange(data.loc[0,0],data.loc[data.shape[0]-1,0])
        for (i,da) in enumerate(act['date']):
             act.loc[i,'act'] = np.sum(data.loc[data[0]==da,1]=='BIKING')+np.sum(data.loc[data[0]==da,1]=='ON_FOOT')
    else:
        print(' no act data')
    acts.append(act)

    ema = pd.DataFrame(columns=['date','stress','mood','energy','focus'], dtype=float)
    if os.path.exists(data_dir+subj+'/emm.csv'):
        data = pd.read_csv(data_dir+subj+'/emm.csv',sep='\t',header=None)
        # convert timestamps to daystamps
        data[0] = np.floor((data[0]+ 3600*float(timezones.loc[timezones[0]==subj,1]))/86400.0)
        # loading into new matrix
        ema['date'] = np.arange(data.loc[0,0],data.loc[data.shape[0]-1,0])
        for (i,da) in enumerate(ema['date']):
            ema.loc[i,'stress'] = np.nanmean(data.loc[data[0]==da,1])
            moods = data.loc[data[0]==da,2]
            ema.loc[i,'mood'] = np.nanmean(moods)
            ema.loc[i,'energy'] = np.nanmean(data.loc[data[0]==da,3])
            ema.loc[i,'focus'] = np.nanmean(data.loc[data[0]==da,4])
    else:
        print(' no ema data')
    emas.append(ema)
    
    ems = pd.DataFrame(columns=['date','duration','quality','daytype'], dtype=float)
    if os.path.exists(data_dir+subj+'/ems.csv'):
        data = pd.read_csv(data_dir+subj+'/ems.csv',sep='\t',header=None)
        # convert timestamps to daystamps
        data[0] = np.floor((data[3]/1000.0+3600*float(timezones.loc[timezones[0]==subj,1]))/86400.0)
        # loading into new matrix
        ems['date'] = np.arange(data.loc[0,0],data.loc[data.shape[0]-1,0])
        for (i,da) in enumerate(ems['date']):
            ems.loc[i,'duration'] = np.nanmean(data.loc[data[0]==da,3]-data.loc[data[0]==da,2])/1000.0
            qual = np.array(data.loc[data[0]==da,5])
            # if multiple entries, only take the first one
            if qual.size>1:
                ems.loc[i,'quality'] = qual[0]
            else:
                ems.loc[i,'quality'] = np.nanmean(qual)
            daytype = data.loc[data[0]==da,6]
            if daytype.size>0:
                ems.loc[i,'daytype'] = daytype2number(daytype.values[0])
            else:
                ems.loc[i,'daytype'] = np.nan
    else:
        print(' no ems data')
    emss.append(ems)
        
    wtr = pd.DataFrame(columns=['date','mean_temp','clear'], dtype=float)
    if os.path.exists(weather_dir+subj+'/wtr.csv') and  os.stat(weather_dir+subj+'/wtr.csv').st_size > 0:
        data = pd.read_csv(weather_dir+subj+'/wtr.csv',sep='\t',header=None)
        # convert timestamps to daystamps
        data[0] = np.floor((data[0]+3600*float(timezones.loc[timezones[0]==subj,1]))/86400.0)
        # loading into new matrix
        wtr['date'] = np.arange(data.loc[0,0],data.loc[data.shape[0]-1,0])
        for (i,da) in enumerate(wtr['date']):
            # wrt.loc[i,'duration'] = np.nanmean(data.loc[data[0]==da,1])/1000.0
            tmptemp = np.array(data.loc[data[0]==da,1])
            tmpclear = np.array(data.loc[data[0]==da, 9])

            # take the mean of multiple entries
            if tmptemp.size>0:
                wtr.loc[i,'mean_temp'] = np.nanmean(tmptemp)
                wtr.loc[i, 'clear'] = np.sum(tmpclear=='Clear');
    else:
        print(' no weather data')
    wtrs.append(wtr)
    
subjects = subjects_edited

0 EW057EV


/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.4/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1 MQ077WG
2 1203725
3 BD921DW
4 1573207
 no act data
5 1578395
6 1464458
7 1135515
8 1183252
9 1521517
10 1210517
11 FM387DI
 no weather data
12 952207
13 1142152
14 984221
15 1553373
16 1244644
 no act data
17 1327952
18 1524496
19 1559190
20 1130955
21 1535103
22 1381257
23 1041667
24 1055808
25 1483186
26 1054952
27 1197009
28 1385032
29 IK750RN
30 IP417XX
31 1564420
32 1288818
33 1571376
34 1567871
35 1328568
36 1374066
37 EW057DI
 no act data
 no ema data
 no ems data
 no weather data
38 1536580
39 XF600QF
40 1208813
41 1114936
42 1165040
 no act data
43 25349
44 1234088
45 827285
46 FJ227MJ
47 1124659
48 1422927
49 1270329
50 1219403
51 345921
52 1367477
53 1505268
54 1579481
55 HH880DL
56 HE593LT
 no weather data
57 WT253RB
58 606449
59 UQ050PR
 no weather data
60 1549226
61 1312819
62 428255
 no act data
63 1013558
64 1425272
65 1428949
 no act data
66 1524947
67 1533327
68 1305690
69 1084240
70 831987
71 1079411
72 1496251
 no weather data
73 1078563
 no act data
74 872451
75 

In [8]:
# aligning the data

data = []
for (i,_) in enumerate(subjects):
    
    a = pd.merge(emas[i],emss[i],on='date',how='outer')
    a = pd.merge(a,acts[i],on='date',how='outer')
    a = pd.merge(a,wtrs[i], on='date', how='outer')
    
    # delayed (-1)
    emas[i]['date'] += 1
    emss[i]['date'] += 1
    acts[i]['date'] += 1
    wtrs[i]['date'] += 1
    emas[i].columns = ['date','stress_prev','mood_prev','energy_prev','focus_prev']
    emss[i].columns = ['date','duration_prev','quality_prev','daytype_prev']
    acts[i].columns = ['date','act_prev']
    wtrs[i].columns = ['date','mean_temp_prev', 'clear_prev']
    a = pd.merge(a,emas[i],on='date',how='outer')
    a = pd.merge(a,emss[i],on='date',how='outer')
    a = pd.merge(a,acts[i],on='date',how='outer')
    a = pd.merge(a,wtrs[i], on='date', how='outer')

    # removing extra columns
    emss[i] = emss[i].drop(['duration_prev','daytype_prev'], axis=1)
    
    # delayed (-2)
    emas[i]['date'] += 1
    emss[i]['date'] += 1
    acts[i]['date'] += 1
    wtrs[i]['date'] += 1
    emas[i].columns = ['date','mood_prev2','stress_prev2','energy_prev2','focus_prev2']
    emss[i].columns = ['date','quality_prev2']
    acts[i].columns = ['date','act_prev2']
    wtrs[i].columns = ['date','mean_temp_prev2', 'clear_prev2']
    a = pd.merge(a,emas[i],on='date',how='outer')
    a = pd.merge(a,emss[i],on='date',how='outer')
    a = pd.merge(a,acts[i],on='date',how='outer')
    a = pd.merge(a,wtrs[i],on='date',how='outer')

    # removing extra columns
    emas[i] = emas[i].drop(['stress_prev2','energy_prev2','focus_prev2'], axis=1)
    
    # delayed (-3)
    emas[i]['date'] += 1
    emss[i]['date'] += 1
    emas[i].columns = ['date','mood_prev3']
    emss[i].columns = ['date','quality_prev3']
    a = pd.merge(a,emas[i],on='date',how='outer')
    a = pd.merge(a,emss[i],on='date',how='outer')

    # delayed (-4) - for mood only
    emas[i]['date'] += 1
    emas[i].columns = ['date','mood_prev4']
    a = pd.merge(a,emas[i],on='date',how='outer')
    
    data.append(a)


In [10]:
# add day of the week and write to disk

import datetime as dt
import calendar

for (i,_) in enumerate(subjects):
    ts =data[i].date*86400
    #tmp =dt.datetime.fromtimestamp().day
    data[i]['dow']=[dt.datetime.fromtimestamp(t).weekday() for t in ts]
    
with open('data.dat','wb') as f:
    pickle.dump([data, subjects], f)
f.close()

In [11]:
data[0]

,date,stress,mood,energy,focus,duration,quality,daytype,act,mean_temp,...,energy_prev2,focus_prev2,quality_prev2,act_prev2,mean_temp_prev2,clear_prev2,mood_prev3,quality_prev3,mood_prev4,dow
0,16783.0,2.000000,1.000000,1.000000,2.000000,15600.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
1,16784.0,2.000000,3.666667,2.000000,2.333333,56400.0,1.0,0.0,103.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,16785.0,2.000000,3.666667,1.666667,1.666667,20400.0,1.0,0.0,31.0,NaN,...,1.000000,2.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1
3,16786.0,2.000000,3.333333,2.000000,2.000000,20700.0,1.0,0.0,28.0,NaN,...,2.000000,2.333333,1.0,103.0,NaN,NaN,2.000000,0.0,NaN,2
4,16787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN,...,1.666667,1.666667,1.0,31.0,NaN,NaN,2.000000,1.0,2.000000,3
5,16788.0,1.333333,2.333333,1.777778,2.222222,30900.0,4.0,2.0,53.0,NaN,...,2.000000,2.000000,1.0,28.0,NaN,NaN,2.000000,1.0,2.000000,4
6,16789.0,2.666667,3.000000,2.333333,2.666667,24600.0,2.0,2.0,41.0,NaN,...,NaN,NaN,NaN,45.0,NaN,NaN,2.000000,1.0,2.000000,5
7,16790.0,2.000000,3.333333,2.333333,2.000000,23100.0,2.0,0.0,48.0,NaN,...,1.777778,2.222222,4.0,53.0,NaN,NaN,NaN,NaN,2.000000,6
8,16791.0,0.666667,0.666667,0.666667,0.666667,65700.0,1.0,0.0,20.0,NaN,...,2.333333,2.666667,2.0,41.0,NaN,NaN,1.333333,4.0,NaN,0
9,16792.0,1.000000,1.500000,1.000000,1.500000,24000.0,2.0,0.0,35.0,NaN,...,2.333333,2.000000,2.0,48.0,NaN,NaN,2.666667,2.0,1.333333,1
